### Requesting and Parsing API response
<p style="max-width:40rem">This notebook will focus on using the datamodel to form a simplified income statement, balance sheet, and cashflows for a company using the api response.</p>

In [186]:
import requests
import pandas as pd
import sys
import json, helpful
pd.set_option('display.float_format', lambda x: '%0.2f' % x)

In [187]:
allTickers = json.loads(open("tickers.json", 'r').read())
ticker = "FB"
cik = allTickers[ticker]['cik']

In [151]:
user_agent = {"user-agent": "www.jonsteeves.dev jonathonsteeves@cmail.carleton.ca"}
url = "https://www.sec.gov/edgar/browse/?CIK=51143"
companyFacts = "https://data.sec.gov/api/xbrl/companyfacts/CIK{:010d}.json".format(cik)
api = "https://data.sec.gov/submissions/CIK0000051143.json"

In [152]:
res = requests.get(companyFacts, headers=user_agent)
res.status_code

200

In [153]:
dc = dict(res.json())

In [154]:
dc['facts'].keys()
len(dc['facts']['us-gaap'])

381

In [155]:
sys.getsizeof(dc["facts"]['us-gaap'])

10292

In [156]:
# dc['facts']['dei']['EntityPublicFloat']

In [157]:
gaap = dc['facts']['us-gaap']

In [158]:
dataModel = json.loads(open('dataModels/dataModelV1.json', 'r').read())
gaapKeys = set(gaap.keys())

In [159]:
t = pd.DataFrame(helpful.keyQuarters(gaap["Revenues"]["units"]["USD"])[-10:])
t.fy = t.start.apply(lambda x: x.split("-")[0])

In [160]:
t

,start,end,val,accn,fy,fp,form,filed,frame
0,2016-04-01,2016-06-30,6436000000,0001326801-17-000038,2016,Q2,10-Q,2017-07-27,CY2016Q2
1,2016-07-01,2016-09-30,7011000000,0001326801-17-000053,2016,Q3,10-Q,2017-11-02,CY2016Q3
2,2016-01-01,2016-12-31,27638000000,0001326801-18-000009,2016,FY,10-K,2018-02-01,CY2016
3,2017-01-01,2017-03-31,8032000000,0001326801-18-000032,2017,Q1,10-Q,2018-04-26,CY2017Q1
4,2017-04-01,2017-06-30,9321000000,0001326801-18-000057,2017,Q2,10-Q,2018-07-26,CY2017Q2
5,2017-07-01,2017-09-30,10328000000,0001326801-18-000067,2017,Q3,10-Q,2018-10-31,CY2017Q3
6,2017-01-01,2017-12-31,40653000000,0001326801-18-000009,2017,FY,10-K,2018-02-01,CY2017
7,2018-01-01,2018-03-31,11966000000,0001326801-18-000032,2018,Q1,10-Q,2018-04-26,CY2018Q1
8,2018-04-01,2018-06-30,13231000000,0001326801-18-000057,2018,Q2,10-Q,2018-07-26,CY2018Q2
9,2018-07-01,2018-09-30,13727000000,0001326801-18-000067,2018,Q3,10-Q,2018-10-31,CY2018Q3


In [161]:
def keyQuarters(ls):
    newls = []
    for frame in ls:
        if('frame' in frame.keys()):
            newls.append(frame)
    return newls

def make_frame(df):
    dataModel = json.loads(open('dataModels/dataModelV3_Income.json', 'r').read())
    incomeModel = dataModel["IncomeStatements"]
    df_model = pd.DataFrame()
    for i in incomeModel:
        try:
            df_model[i] = consolidate(keyQuarters(df[incomeModel[i]]["units"]["USD"]))
        except:
            print(df[incomeModel[i]]["units"]["USD"])
            df_model[i] = None
        # return pd.DataFrame(keyQuarters(df[dataModel[i]]["units"]["USD"]))
    
        # print(df[dataModel[i]]["units"]["USD"][-1])
        # return pd.DataFrame(df[dataModel[i]]["units"]["USD"])
    return df_model

def consolidate(ls):
    frame = pd.DataFrame(ls)
    frame.fy = frame.end.apply(lambda x: x.split("-")[0])
    # frame.fy = frame.end.apply(lambda x: x.split("-")[0])
    frame = frame[["fy", "val", "frame", "form"]]
    frame = pd.concat([frame, make_q4(frame)])
    # frame["date"] = frame["start"] + ', ' + frame["end"]
    
    frame["frame2"] = frame["frame"].transform(lambda x: x[2:])
    # frame = frame.set_index('frame')
    # frame.drop(["start", "end", "accn", "filed"], axis=1, inplace=True)
    return frame[frame["form"] != "10-K"].sort_values(["fy", "form"]).set_index("frame")["val"]

def make_q4(arr):
    crs = pd.crosstab(arr["fy"], arr["form"], values=arr["val"], aggfunc="sum")
    q = crs["10-K"] - crs["10-Q"]
    out = pd.DataFrame(q.dropna(), columns=["val"]).reset_index()
    out["frame"] = out["fy"].transform(lambda x: f"CY{x}Q4")
    out["form"] = "10-Q"
    return out

In [162]:
dataModel = json.loads(open('dataModel/dataModelV3_Income.json', 'r').read())
incomeModel = dataModel["IncomeStatements"]

In [163]:
dataModel

{'IncomeStatements': {'Total revenue': 'Revenues',
  'Total cost': 'CostOfRevenue',
  'Gross profit': 'GrossProfit',
  'Selling, general and administrative': 'SellingGeneralAndAdministrativeExpense',
  'Research, development and engineering': 'ResearchAndDevelopmentExpense',
  'Interest expense': 'InterestExpense',
  'Income before taxes': 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
  'Net income': 'NetIncomeLoss'},
 'BalanceSheet': {'Total current assets': 'AssetsCurrent',
  'Total assets': 'Assets',
  'Total current liabilities': 'LiabilitiesCurrent',
  'Total liabilities': 'Liabilities',
  'Retained earnings': 'RetainedEarningsAccumulatedDeficit',
  'Total equity': 'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest',
  'Total liabilities and equity': 'LiabilitiesAndStockholdersEquity'}}

In [164]:
for i in incomeModel:
    print(incomeModel[i])

Revenues
CostOfRevenue
GrossProfit
SellingGeneralAndAdministrativeExpense
ResearchAndDevelopmentExpense
InterestExpense
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest
NetIncomeLoss


In [181]:
p = pd.DataFrame(gaap).drop("units")

In [184]:
p.T.reset_index()

,index,label,description
0,AccountsPayableAndOtherAccruedLiabilitiesCurrent,Accounts Payable and Other Accrued Liabilities...,Amount of liabilities incurred to vendors for ...
1,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...
2,AccountsPayableOtherCurrent,"Accounts Payable, Other, Current",Amount of obligations incurred classified as o...
3,AccountsPayableTradeCurrent,"Accounts Payable, Trade, Current",Carrying value as of the balance sheet date of...
4,AccountsReceivableNetCurrent,"Accounts Receivable, after Allowance for Credi...","Amount, after allowance for credit loss, of ri..."
...,...,...,...
376,ContractualObligationDueInSecondYear,"Contractual Obligation, to be Paid, Year Two",Amount of contractual obligation to be paid in...
377,ContractualObligationDueInThirdYear,"Contractual Obligation, to be Paid, Year Three",Amount of contractual obligation to be paid in...
378,ContractualObligationFutureMinimumPaymentsDueR...,"Contractual Obligation, to be Paid, Remainder ...",Amount of contractual obligation to be paid in...
379,GoodwillOtherIncreaseDecrease,"Goodwill, Other Increase (Decrease)","Amount of increase (decrease), classified as o..."


In [185]:
gaap["OperatingIncomeLoss"]

{'label': 'Operating Income (Loss)',
 'description': 'The net result for the period of deducting operating expenses from operating revenues.',
 'units': {'USD': [{'start': '2010-01-01',
    'end': '2010-12-31',
    'val': 1032000000,
    'accn': '0001326801-13-000003',
    'fy': 2012,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2013-02-01',
    'frame': 'CY2010'},
   {'start': '2011-01-01',
    'end': '2011-06-30',
    'val': 795000000,
    'accn': '0001193125-12-325997',
    'fy': 2012,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2012-07-31'},
   {'start': '2011-04-01',
    'end': '2011-06-30',
    'val': 407000000,
    'accn': '0001193125-12-325997',
    'fy': 2012,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2012-07-31',
    'frame': 'CY2011Q2'},
   {'start': '2011-01-01',
    'end': '2011-09-30',
    'val': 1209000000,
    'accn': '0001326801-12-000006',
    'fy': 2012,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2012-10-24'},
   {'start': '2011-07-01',
    

In [173]:
cht = make_frame(gaap)
# cht.T.to_excel("example_table2.xlsx")

KeyError: 'GrossProfit'

In [139]:
cht.tail()

,Total revenue,Total cost,Gross profit,"Selling, general and administrative","Research, development and engineering",Interest expense,Income before taxes,Net income
frame,,,,,,,,
CY2021Q1,13187000000.00,6160000000.00,7027000000.00,4688000000.00,1616000000.00,280000000.00,244000000.00,955000000.00
CY2021Q2,18745000000.00,9741000000.00,9004000000.00,5334000000.00,1657000000.00,281000000.00,1552000000.00,1325000000.00
CY2021Q3,17618000000.00,9447000000.00,8171000000.00,4860000000.00,1621000000.00,291000000.00,1319000000.00,1130000000.00
CY2021Q4,7800000000.00,517000000.00,7284000000.00,3863000000.00,1594000000.00,303000000.00,1722000000.00,2333000000.00
CY2022Q1,14197000000.00,6862000000.00,7335000000.00,4597000000.00,1679000000.00,311000000.00,623000000.00,733000000.00


In [120]:
gaap["Revenues"]["units"]["USD"]

[{'start': '2007-01-01',
  'end': '2007-12-31',
  'val': 98786000000,
  'accn': '0001047469-10-001151',
  'fy': 2009,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2010-02-23',
  'frame': 'CY2007'},
 {'start': '2008-01-01',
  'end': '2008-06-30',
  'val': 51322000000,
  'accn': '0001104659-09-045198',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-07-28'},
 {'start': '2008-04-01',
  'end': '2008-06-30',
  'val': 26820000000,
  'accn': '0001104659-09-045198',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-07-28',
  'frame': 'CY2008Q2'},
 {'start': '2008-01-01',
  'end': '2008-09-30',
  'val': 76623000000,
  'accn': '0001104659-09-060554',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-10-27'},
 {'start': '2008-07-01',
  'end': '2008-09-30',
  'val': 25302000000,
  'accn': '0001104659-09-060554',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-10-27',
  'frame': 'CY2008Q3'},
 {'start': '2008-01-01',
  'end': '2008-12-31',
  'va

In [121]:
pd.DataFrame(gaap["Assets"]["units"]["USD"])

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,109524000000,0001104659-09-045198,2009,Q2,10-Q,2009-07-28,NaN
1,2008-12-31,109524000000,0001104659-09-060554,2009,Q3,10-Q,2009-10-27,NaN
2,2008-12-31,109524000000,0001047469-10-001151,2009,FY,10-K,2010-02-23,NaN
3,2008-12-31,109524000000,0001047469-11-001117,2010,FY,10-K,2011-02-22,CY2008Q4I
4,2009-06-30,103655000000,0001104659-09-045198,2009,Q2,10-Q,2009-07-28,CY2009Q2I
...,...,...,...,...,...,...,...,...
114,2021-06-30,146814000000,0001558370-21-009351,2021,Q2,10-Q,2021-07-27,CY2021Q2I
115,2021-09-30,144214000000,0001558370-21-014734,2021,Q3,10-Q,2021-11-05,CY2021Q3I
116,2021-12-31,132001000000,0001558370-22-001584,2021,FY,10-K,2022-02-22,NaN
117,2021-12-31,132001000000,0001558370-22-005983,2022,Q1,10-Q,2022-04-26,CY2021Q4I


In [619]:
frame = pd.DataFrame(keyQuarters(gaap["Revenues"]["units"]["USD"]))
frame = frame[["fy", "val", "frame", "form"]]
# frame = frame.append(make_q4(frame))
frame
# make_q4(consolidate())

,fy,val,frame,form
0,2009,98786000000,CY2007,10-K
1,2009,26820000000,CY2008Q2,10-Q
2,2009,25302000000,CY2008Q3,10-Q
3,2010,103630000000,CY2008,10-K
4,2010,21711000000,CY2009Q1,10-Q
5,2010,23250000000,CY2009Q2,10-Q
6,2010,23566000000,CY2009Q3,10-Q
7,2011,95758000000,CY2009,10-K
8,2011,22857000000,CY2010Q1,10-Q
9,2011,23724000000,CY2010Q2,10-Q


In [637]:
gaap["Assets"]["units"]["USD"]

[{'end': '2008-12-31',
  'val': 109524000000,
  'accn': '0001104659-09-045198',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-07-28'},
 {'end': '2008-12-31',
  'val': 109524000000,
  'accn': '0001104659-09-060554',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-10-27'},
 {'end': '2008-12-31',
  'val': 109524000000,
  'accn': '0001047469-10-001151',
  'fy': 2009,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2010-02-23'},
 {'end': '2008-12-31',
  'val': 109524000000,
  'accn': '0001047469-11-001117',
  'fy': 2010,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2011-02-22',
  'frame': 'CY2008Q4I'},
 {'end': '2009-06-30',
  'val': 103655000000,
  'accn': '0001104659-09-045198',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-07-28',
  'frame': 'CY2009Q2I'},
 {'end': '2009-09-30',
  'val': 103675000000,
  'accn': '0001104659-09-060554',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-10-27',
  'frame': 'CY2009Q3I'},
 {'end': '2009-1

In [634]:
consolidate(keyQuarters(gaap["Revenues"]["units"]["USD"]))

C:\Users\jonat\AppData\Local\Temp\ipykernel_12040\4174423929.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frame = frame.append(make_q4(frame))


frame
CY2007      98786000000.00
CY2008     103630000000.00
CY2009      95758000000.00
CY2008Q2    26820000000.00
CY2008Q3    25302000000.00
                 ...      
CY2021Q1    13187000000.00
CY2021Q2    18745000000.00
CY2021Q3    17618000000.00
CY2021Q4     7800000000.00
CY2022Q1    14197000000.00
Name: val, Length: 69, dtype: float64

In [540]:
t["frame2"] = t["frame"].transform(lambda x: x[2:])

In [533]:
s = pd.DataFrame(consolidate(keyQuarters(gaap["Revenues"]["units"]["USD"])))

In [535]:
s

,val
frame,
CY2007,98786000000
CY2008Q2,26820000000
CY2008Q3,25302000000
CY2008,103630000000
CY2009Q1,21711000000
CY2009Q2,23250000000
CY2009Q3,23566000000
CY2009,95758000000
CY2010Q1,22857000000


In [122]:
make_q4(t)["fy"].transform(lambda x: f"CY{x}Q4")

0    CY2020Q4
1    CY2021Q4
Name: fy, dtype: object

In [140]:
chart = make_frame(gaap)

In [124]:
chart.T.to_excel("example_table.xlsx")

In [149]:
chart.tail(20).T

frame,CY2017Q2,CY2017Q3,CY2017Q4,CY2018Q1,CY2018Q2,CY2018Q3,CY2018Q4,CY2019Q1,CY2019Q2,CY2019Q3,CY2019Q4,CY2020Q1,CY2020Q2,CY2020Q3,CY2020Q4,CY2021Q1,CY2021Q2,CY2021Q3,CY2021Q4,CY2022Q1
Total revenue,19289000000.00,19153000000.00,22542000000.00,19072000000.00,20003000000.00,18756000000.00,21760000000.00,18182000000.00,19161000000.00,18028000000.00,2343000000.00,17571000000.00,18123000000.00,17560000000.00,1925000000.00,13187000000.00,18745000000.00,17618000000.00,7800000000.00,14197000000.00
Total cost,10321000000.00,10172000000.00,11492000000.00,10825000000.00,10804000000.00,9953000000.00,11073000000.00,10139000000.00,10151000000.00,9692000000.00,-3801000000.00,9649000000.00,9423000000.00,9130000000.00,-3888000000.00,6160000000.00,9741000000.00,9447000000.00,517000000.00,6862000000.00
Gross profit,8968000000.00,8981000000.00,11050000000.00,8247000000.00,9199000000.00,8803000000.00,10687000000.00,8043000000.00,9010000000.00,8336000000.00,6144000000.00,7922000000.00,8700000000.00,8430000000.00,5813000000.00,7027000000.00,9004000000.00,8171000000.00,7284000000.00,7335000000.00
"Selling, general and administrative",5033000000.00,4606000000.00,5014000000.00,5445000000.00,4857000000.00,4363000000.00,4701000000.00,4691000000.00,5456000000.00,5024000000.00,3553000000.00,5955000000.00,5248000000.00,4647000000.00,4711000000.00,4688000000.00,5334000000.00,4860000000.00,3863000000.00,4597000000.00
"Research, development and engineering",1436000000.00,1291000000.00,1379000000.00,1405000000.00,1364000000.00,1252000000.00,1358000000.00,1433000000.00,1407000000.00,1553000000.00,1517000000.00,1625000000.00,1582000000.00,1515000000.00,1540000000.00,1616000000.00,1657000000.00,1621000000.00,1594000000.00,1679000000.00
Interest expense,147000000.00,168000000.00,165000000.00,165000000.00,173000000.00,191000000.00,194000000.00,210000000.00,348000000.00,432000000.00,354000000.00,326000000.00,323000000.00,323000000.00,316000000.00,280000000.00,281000000.00,291000000.00,303000000.00,311000000.00
Income before taxes,2443000000.00,3065000000.00,4468000000.00,1136000000.00,2776000000.00,2996000000.00,4434000000.00,1883000000.00,2768000000.00,1522000000.00,1033000000.00,-49000000.00,1571000000.00,1827000000.00,-777000000.00,244000000.00,1552000000.00,1319000000.00,1722000000.00,623000000.00
Net income,2331000000.00,2726000000.00,-1054000000.00,1679000000.00,2404000000.00,2694000000.00,1951000000.00,1591000000.00,2498000000.00,1672000000.00,3670000000.00,1175000000.00,1361000000.00,1698000000.00,1356000000.00,955000000.00,1325000000.00,1130000000.00,2333000000.00,733000000.00


In [128]:
gaap["NetIncomeLoss"]

{'label': 'Net Income (Loss) Attributable to Parent',
 'description': 'The portion of profit or loss for the period, net of income taxes, which is attributable to the parent.',
 'units': {'USD': [{'start': '2015-01-01',
    'end': '2015-12-31',
    'val': 13190000000,
    'accn': '0001047469-18-001117',
    'fy': 2017,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2018-02-27',
    'frame': 'CY2015'},
   {'start': '2016-01-01',
    'end': '2016-03-31',
    'val': 2014000000,
    'accn': '0001104659-17-025881',
    'fy': 2017,
    'fp': 'Q1',
    'form': '10-Q',
    'filed': '2017-04-25',
    'frame': 'CY2016Q1'},
   {'start': '2016-01-01',
    'end': '2016-06-30',
    'val': 4518000000,
    'accn': '0001104659-17-046808',
    'fy': 2017,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2017-07-25'},
   {'start': '2016-04-01',
    'end': '2016-06-30',
    'val': 2504000000,
    'accn': '0001104659-17-046808',
    'fy': 2017,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2017-07-

In [330]:
t[(t["end"] < "2021-12-31") & (t["start"] >= "2021-01-01")]["val"].sum()


49550000000

In [314]:
consolidate(keyQuarters(gaap["Revenues"]["units"]["USD"]))

,val
frame,
CY2007,98786000000
CY2008Q2,26820000000
CY2008Q3,25302000000
CY2008,103630000000
CY2009Q1,21711000000
CY2009Q2,23250000000
CY2009Q3,23566000000
CY2009,95758000000
CY2010Q1,22857000000


In [416]:
keyQuarters(gaap["Revenues"]["units"]["USD"])[-10:]

[{'start': '2019-01-01',
  'end': '2019-12-31',
  'val': 57714000000,
  'accn': '0001558370-22-001584',
  'fy': 2021,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2022-02-22',
  'frame': 'CY2019'},
 {'start': '2020-01-01',
  'end': '2020-03-31',
  'val': 17571000000,
  'accn': '0001558370-21-004922',
  'fy': 2021,
  'fp': 'Q1',
  'form': '10-Q',
  'filed': '2021-04-27',
  'frame': 'CY2020Q1'},
 {'start': '2020-04-01',
  'end': '2020-06-30',
  'val': 18123000000,
  'accn': '0001558370-21-009351',
  'fy': 2021,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2021-07-27',
  'frame': 'CY2020Q2'},
 {'start': '2020-07-01',
  'end': '2020-09-30',
  'val': 17560000000,
  'accn': '0001558370-21-014734',
  'fy': 2021,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2021-11-05',
  'frame': 'CY2020Q3'},
 {'start': '2020-01-01',
  'end': '2020-12-31',
  'val': 55179000000,
  'accn': '0001558370-22-001584',
  'fy': 2021,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2022-02-22',
  'frame': 'CY2020'},
 {'start'